<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/cleaning_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregating script

## Importing libraries

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from excel to a pandas dataframe
#import io
#df1 = pd.read_csv(io.BytesIO(uploaded1['Script_202208170953.xlsx']))

In [ ]:
# Storing loaded data from csv to a pandas dataframe
import io
df1 = pd.read_csv(io.BytesIO(uploaded1['Script_202208171546.csv']), sep='|', engine='python')

In [ ]:
# Selecting columns
custom_list_1 = df1.columns.tolist()

In [ ]:
# Checking the dataframe shape
df1.tail(1)

## Exploring Data

In [ ]:
# Interacting with list of columns
@interact(Column_name = custom_list_1, Category=df1['EVENT_TYPE'].unique(), Percentage = [False, True])
def explore_columns(Column_name, Category, Percentage):
  return pd.DataFrame(df1[df1['EVENT_TYPE']==Category][Column_name].value_counts(normalize=Percentage, dropna=False))

## Aggregating Data

In [ ]:
# Selecting columns for aggregation
custom_list_2 = ['', '', '', ''] # => Add column names for slicing here.

In [ ]:
# Main data aggregation
df1_a = df1.groupby(by=custom_list_2, dropna=False).agg(CHURN_COUNT = ('CUSTOMER_ID', 'count')).reset_index() # => Note: Dropna=False to avoid dropping data if group keys contain NA values.

In [ ]:
# Creating list of aggregated fields
df1_list = [(pd.DataFrame(df1[i].value_counts(dropna=False))) for i in custom_list_1]

## Exporting results to local drive

In [ ]:
# Exporting main excel file
with pd.ExcelWriter('Script_202208170859.xlsx', engine='openpyxl') as writer:
  df1_a.to_excel(writer, sheet_name='Bedriftsliste', index=False)
files.download('Script_202208170859.xlsx')

In [ ]:
# Exporting list of excel sheets
with pd.ExcelWriter('Script_202208170859.xlsx', engine='openpyxl') as writer:
  for i in range(len(df1_list)-1):
    df1_list[i].to_excel(writer, sheet_name=f'{df1_list[i].columns[0]}', index=True)
files.download('Script_202208170859.xlsx')